<a href="https://colab.research.google.com/github/MarinaWolters/Coding-Tracker/blob/master/Part2_Spark_on_Cluster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!apt install libkrb5-dev
!wget https://www-us.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz
!pip install findspark
!pip install sparkmagic
!pip install pyspark
! pip install pyspark --user
! pip install seaborn --user
! pip install plotly --user
! pip install imageio --user
! pip install folium --user

In [ ]:
%%capture
!apt update
!apt install gcc python-dev libkrb5-dev

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

import os

spark = SparkSession.builder.appName('Graphs-HW2').getOrCreate()


In [ ]:
import numpy as np
import pandas as pd
import matplotlib

#misc
import gc
import time
import warnings


#graph section
import networkx as nx
#import heapq  # for getting top n number of things from list,dict
import pandas as pd
import numpy as np

# JSON parsing
import json

# HTML parsing
from lxml import etree
import urllib

# SQLite RDBMS
import sqlite3

# Time conversions
import time

# Parallel processing
# import swifter

# NoSQL DB
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

import os
os.environ['SPARK_HOME'] = "/content/spark-3.0.1-bin-hadoop3.2" #change this here
import pyspark
from pyspark.sql import SQLContext

In [ ]:
try:
    if(spark == None):
        spark = SparkSession.builder.appName('Initial').getOrCreate()
        sqlContext=SQLContext(spark)
except NameError:
    spark = SparkSession.builder.appName('Initial').getOrCreate()
    sqlContext=SQLContext(spark)


In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='11lwBibxX7PYGgOfHU25_dDDDsPX1Pt0Y',
                                    dest_path='/content/yelp_business.csv')


In [ ]:
yelp_business_sdf = spark.read.format("csv").option("header", "true").load("yelp_business.csv")
yelp_business_sdf.createOrReplaceTempView('yelp_business')


In [ ]:
yelp_business_sdf.show(5)

+--------------------+--------------------+------------+--------------------+--------------+-----+-----------+----------+------------+-----+------------+-------+--------------------+
|         business_id|                name|neighborhood|             address|          city|state|postal_code|  latitude|   longitude|stars|review_count|is_open|          categories|
+--------------------+--------------------+------------+--------------------+--------------+-----+-----------+----------+------------+-----+------------+-------+--------------------+
|FYWN1wneV18bWNgQj...|    Dental by Design|        null|4855 E Warner Rd,...|     Ahwatukee|   AZ|      85044|33.3306902|-111.9785992|  4.0|          22|      1|Dentists;General ...|
|He-G7vWjzVUysIKrf...| Stephen Szabo Salon|        null|  3101 Washington Rd|      McMurray|   PA|      15317|40.2916853| -80.1048999|  3.0|          11|      1|Hair Stylists;Hai...|
|KQPW8lFf1y5BT2Mxi...|Western Motor Veh...|        null|6025 N 27th Ave, ...|       P

In [ ]:
avg_reviews_by_city_sdf = spark.sql('select city, avg(stars) as avg_rating from yelp_business yb '\
                                             'group by city')

avg_reviews_by_city_sdf.explain()

== Physical Plan ==
*(2) HashAggregate(keys=[city#129], functions=[avg(cast(stars#134 as double))])
+- Exchange hashpartitioning(city#129, 200), true, [id=#98]
   +- *(1) HashAggregate(keys=[city#129], functions=[partial_avg(cast(stars#134 as double))])
      +- FileScan csv [city#129,stars#134] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/content/yelp_business.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<city:string,stars:string>




In [ ]:
same_city_sdf = spark.sql('select b1.name, b2.name from yelp_business b1 join yelp_business b2 '\
                                             ' on b1.city = b2.city and b1.name <> b2.name')

same_city_sdf.show(5)

+----------------+--------------------+
|            name|                name|
+----------------+--------------------+
|Dental by Design|      My Wine Cellar|
|Dental by Design|Florencia Pizza B...|
|Dental by Design|           Hi-Health|
|Dental by Design|       Barro's Pizza|
|Dental by Design|Healthy and Clean...|
+----------------+--------------------+
only showing top 5 rows



In [ ]:
same_city_sdf.explain()

== Physical Plan ==
*(2) Project [name#126, name#427]
+- *(2) BroadcastHashJoin [city#129], [city#430], Inner, BuildRight, NOT (name#126 = name#427)
   :- *(2) Project [name#126, city#129]
   :  +- *(2) Filter (isnotnull(city#129) AND isnotnull(name#126))
   :     +- FileScan csv [name#126,city#129] Batched: false, DataFilters: [isnotnull(city#129), isnotnull(name#126)], Format: CSV, Location: InMemoryFileIndex[file:/content/yelp_business.csv], PartitionFilters: [], PushedFilters: [IsNotNull(city), IsNotNull(name)], ReadSchema: struct<name:string,city:string>
   +- BroadcastExchange HashedRelationBroadcastMode(List(input[1, string, true])), [id=#550]
      +- *(1) Project [name#427, city#430]
         +- *(1) Filter (isnotnull(city#430) AND isnotnull(name#427))
            +- FileScan csv [name#427,city#430] Batched: false, DataFilters: [isnotnull(city#430), isnotnull(name#427)], Format: CSV, Location: InMemoryFileIndex[file:/content/yelp_business.csv], PartitionFilters: [], PushedFilt